<a href="https://colab.research.google.com/github/shahabbadihi/IT/blob/master/content-based-evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sklearn
import pandas as pd
import requests
import io

file_url = "https://raw.githubusercontent.com/shahabbadihi/IT/master/PreprocessedDataset2.csv"
response = requests.get(file_url)

dest = 'local-file.csv'

with open(dest, "wb") as filee:
  filee.write(response.content)

df = pd.read_csv(dest)
df=df.fillna('')
df.head(74998)


,product_id,product_title,title_en,user_id,likes,dislikes,verification_status,recommend,title,comment,advantages,disadvantages,cart_id
0,559850,خاک و کود تی تی بسته 10 کیلوگرمی,TC,466132,2,0,verified,not_recommended,خاک عاری از مواد مقوی,اصلا خاک خوبی نبود. کلی سنگ های درشت داشت و اص...,"[""ندارد""]","[""پر از سنگ ریزه ""]",1
1,112946,توستر فلر مدل TO 220,HA,466139,2,0,verified,not_recommended,خراب شدن اهرم پایین برنده,بعد از دو و نیم سال استفاده، اهرم پایین دهنده ...,"[""زیبایی، سبکی و اندازه مناسب توستر""]","[""خراب شدن اهرم پایین برنده""]",2
2,607671,هدست واقعیت مجازی ایکس پی پروداکت مدل VR BOX,AC,466160,1,2,verified,no_idea,قیمت پایین فدای کیفیت .....,کیفیت لنزش پایینه شرکت فروشنده گارانتی تعویض ن...,"[""ارزون""]","[""کیفیت پایین لنز""]",3
3,757892,کیف پول زنانه آیسا مدل KZP-1,FA,466160,1,0,verified,not_recommended,کیفیت ساخت پایین,اصلا با قیمت فروش ویژه هم ارزش خرید نداره_x000...,,"[""کیفیت ساخت پایین\r"",""چسب کاری داخل و بیرون""]",3
4,601446,اسپیکر قابل حمل مدل ip085 مناسب برای آیفون و ...,IT,466160,2,2,verified,no_idea,بدرد ساختن اسپیکر پرتابل میخوره,دوستان من این مدل برای ساخت اسپیکر پرتابل گرفت...,"[""کیفیت قابل قبول برد و آمپیلی فایر داخلی\r"",""...","[""نداشتن بلوتوث""]",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74993,329655,کفش راحتی زنانه لی نینگ مدل Bubble Face,FA,7093994,0,0,verified,\N,حتما بخرید,درسته قیمتش زیاده ولی کیفیتش عالیه _x000D_\nمن...,,,74993
74994,624592,فندک مدادی واته 8021005,HW,7094033,1,0,verified,\N,ارزش خرید داره,خودتون باید ابتکار داشته باشید کمی ارتقاش بدید...,"[""خوش دسته برای جوش های ظریف""]","[""نازل سوختش راحت خراب میشه""]",74995
74995,222488,اسپیکر مارلی مدل CHANT MINI,IT,7094148,0,1,verified,\N,عالی پیشنهاد میکنم,واقعا خیلی خوبه الان ربیق هارمن کاردن خیلی خوب...,"[""کیفیت بالا ""]","[""هیچی""]",74996
74996,48802,نرم افزار جامع جاوا اسکریپت,MA,7094364,0,0,verified,no_idea,خواب میره ادم به خدا,وقتی داره آموزش میده خواب میری به نظرم که خوب...,,"[""کند بودن تایپ\r"",""کند بودن اموزش\r"",""ضعیف بو...",74997


In [7]:
!pip install hazm
from pandas.core.frame import DataFrame
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import random
from scipy.sparse import csr_matrix
from __future__ import unicode_literals
from hazm import *

dff = DataFrame()
dff["user_id"] = df["user_id"]
dff["product_id"]= df["product_id"]
dff["content"] = df["comment"].str.cat(df["advantages"], sep = "-").str.cat(df["disadvantages"], sep = "-").str.cat(df["title"], sep = "-").str.cat(df["product_title"], sep = "-")


dff['content'].head(100)
stemmer = Stemmer()
splitter = TokenSplitter()
normalizer = Normalizer()
for index, row in dff.iterrows():

  c = row['content']
  c = normalizer.affix_spacing(c)
  c = word_tokenize (c)

  for i in range(len(c)):
    c[i] = stemmer.stem(c[i])
  
  dff.at[index, 'content'] = ' '.join(c)

dff['content'].head(100)
file_url = "https://raw.githubusercontent.com/shahabbadihi/IT/master/fa_stop_words.txt"
response = requests.get(file_url)

stop_words = []
with open("stopwords.txt", "wb") as filee:
  filee.write(response.content)

with open("stopwords.txt", "r") as filee:  
  for line in filee:
    stop_words.append(line)
stop_words = set(stop_words)

tf = TfidfVectorizer(analyzer='word',ngram_range=(1,3),min_df=0, stop_words=stop_words)
tfidf_matrix = tf.fit_transform(dff.loc[0:100000, 'content'].values.astype('U'))

print(tfidf_matrix)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آباد', 'آخ', 'آخر', 'آخرها', 'آخه', 'آدمهاست', 'آرام', 'آره', 'آری', 'آزادانه', 'آسان', 'آسانی', 'آسیب', 'آشنایند', 'آشکارا', 'آقا', 'آقای', 'آقایان', 'آمد', 'آمدن', 'آمده', 'آمرانه', 'آن', 'آنان', 'آنانی', 'آنجا', 'آنرا', 'آنطور', 'آنقدر', 'آنكه', 'آنها', 'آنهاست', 'آنچنان', 'آنچه', 'آنکه', 'آنگاه', 'آهان', 'آهای', 'آور', 'آورد', 'آوردن', 'آورده', 'آوه', 'آيد', 'آی', 'آیا', 'آید', 'آیند', 'اتفاقا', 'اثر', 'اجراست', 'احتراما', 'احتمالا', 'احیانا', 'اخیر', 'اخیرا', 'اری', 'از', 'ازاین', 'ازجمله', 'ازش', 'اساسا', 'است', 'استفاد', 'استفاده', 'اسلامی', 'اش', 'اشتباها', 'اشکارا', 'اصلا', 'اصولا', 'اطلاعند', 'اعلام', 'اغلب', 'افزود', 'افسوس', 'اقل', 'اقلیت', 'اكنون', 'الا', 'الاسف', 'الان', 'البت', 'البته', 'الظاهر', 'الهی', 'الی', 'ام', 'اما', 'امروز', 'امروزه', 'امسال', 'امشب', 'امور', 'امیدوارم

  (0, 770264)	0.13056620232084207
  (0, 3411862)	0.1386999773002651
  (0, 1183914)	0.13394202394799862
  (0, 2451816)	0.1386999773002651
  (0, 2508001)	0.1386999773002651
  (0, 288343)	0.1386999773002651
  (0, 2027125)	0.13394202394799862
  (0, 1183649)	0.1386999773002651
  (0, 1679837)	0.1386999773002651
  (0, 1822994)	0.1386999773002651
  (0, 280314)	0.1386999773002651
  (0, 3071324)	0.13394202394799862
  (0, 2689555)	0.13056620232084207
  (0, 2667218)	0.1386999773002651
  (0, 2097310)	0.1386999773002651
  (0, 356966)	0.1386999773002651
  (0, 1333588)	0.1386999773002651
  (0, 1409578)	0.1386999773002651
  (0, 1822939)	0.1386999773002651
  (0, 3306399)	0.1386999773002651
  (0, 2667958)	0.1386999773002651
  (0, 1253155)	0.13394202394799862
  (0, 1183504)	0.1386999773002651
  (0, 355381)	0.1386999773002651
  (0, 8383)	0.11110763072460639
  :	:
  (99999, 2114631)	0.32592456241818274
  (99999, 1636970)	0.05929784028969159
  (99999, 2197530)	0.1767189187492391
  (99999, 702698)	0.083967932

In [8]:
def recommendations(index):
  cos_sim = (cosine_similarity(tfidf_matrix[index],tfidf_matrix))
  x=int( df.loc[[index]].product_id)
  sorted_sim = pd.Series(cos_sim[0]).sort_values(ascending = False)
  # print(sorted_sim)
  top_10 = list(sorted_sim.iloc[1:11].index)
  return top_10

recommendations(500)

500      1.000000
20767    0.166775
8782     0.151400
65723    0.106661
5252     0.090848
           ...   
20109    0.000000
20100    0.000000
81440    0.000000
98347    0.000000
67192    0.000000
Length: 100000, dtype: float64


[20767, 8782, 65723, 5252, 99472, 82859, 87135, 89819, 39244, 9523]

In [9]:
import random
likes= 0
for i in range(3):
  random_number=random.randint(0,100000)
  print('عنوان کالا:')
  print(df.loc[[random_number]].product_title)
  print('عناوین کالا های پیشنهادی : ')
  for recommend in recommendations(random_number):
    print(df.loc[[recommend]].product_title)
  print('از چه تعداد از پیشنهادات رضایت داشتید ؟ ')
  likes = likes + int(input())
print ('recall is: ' + str(likes/30))

عنوان کالا:
57797    آون توستر 50 لیتری  وینزو مدل S-250
Name: product_title, dtype: object
عناوین کالا های پیشنهادی : 
57797    1.000000
66600    0.353673
60121    0.253381
83946    0.163397
73291    0.107422
           ...   
16965    0.000000
93676    0.000000
52936    0.000000
52918    0.000000
0        0.000000
Length: 100000, dtype: float64
66600    آون توستر 50 لیتری  وینزو مدل Lopez-T
Name: product_title, dtype: object
60121    آون توستر 50 لیتری  وینزو مدل Lopez-T
Name: product_title, dtype: object
83946    دستبند طلا 18 عیار ماهک مدل MB0314
Name: product_title, dtype: object
73291    آون توستر Volker مدل 4216
Name: product_title, dtype: object
39008    آون توستر 50 لیتری  وینزو مدل Lopez-T
Name: product_title, dtype: object
98847    آون توستر تکنو مدل Te-90
Name: product_title, dtype: object
49808    آون توستر هانس مدل HT35196
Name: product_title, dtype: object
73165    زیر دکمه دار آستین کوتاه بیبی دی طرح پاندا
Name: product_title, dtype: object
94206    نگهدارنده لاک کد 121